In [26]:
import ccxt
import pandas as pd
import credentials2
import talib as ta
import numpy as np
import datetime
from pytz import timezone
import schedule
import time

exchange = ccxt.binanceusdm({
    'enableRateLimit': True,
    'apiKey': credentials2.api_key,
    'secret': credentials2.api_secret,
    'defaultType': 'future'
})

# Variables
trend = '1d'
exec_tf = '1m'
rsi_period = 2
rsi_overbought = 90
rsi_oversold = 10
sma_period = 200
start_trade = 21  # New York session start time (9:00 PM UTC)
end_trade = 11  # London session end time (11:00 AM UTC)
leverage = 5
marginMode = 'cross'
in_position = False

# def fetch_open_positions():
#     in_position = False
#     positions = exchange.fetchPositions(symbols = ['ETH/USDT'])[-1]['info']['unRealizedProfit']
#     if positions != 0.00000000:
#         print(f' ETH/USDT {positions}')
#         in_position = True
#         return in_position
#     return in_position
trading_symbols = ['ETH/USDT', 'SOL/USDT', 'MATIC/USDT', 'BNB/USDT', 'ADA/USDT', 'GALA/USDT', 'XRP/USDT', 'TRX/USDT', 'LTC/USDT', 'DOT/USDT', 'AVAX/USDT', 'LINK/USDT']
for symbol in trading_symbols:

    def MeanReversionBot():
        global in_position

        pos = float(exchange.fetchPositions(symbols=[symbol])[-1]['info']['entryPrice'])
        if pos > 0:
            in_position = True

        daily_bars = exchange.fetchOHLCV(symbol=symbol, timeframe=trend, limit=sma_period)
        daily_close_price = [bar[4] for bar in daily_bars]
        daily_sma = ta.SMA(np.array(daily_close_price), sma_period)

        if daily_close_price[-1] > daily_sma[-1]:
            exec_tf_bars = exchange.fetchOHLCV(symbol=[symbol], timeframe=exec_tf, limit=sma_period)
            exec_tf_close_price = [bar[4] for bar in exec_tf_bars]
            exec_tf_rsi = ta.RSI(np.array(exec_tf_close_price), rsi_period)
            print(f'{symbol} is bullish, | RSI {exec_tf_rsi[-1]}')

            if not in_position:
                if exec_tf_rsi[-1] < rsi_oversold:
                    exchange.setLeverage(leverage = leverage, symbol=[symbol])
                    exchange.setMarginMode(marginMode = marginMode, symbol=[symbol])
                    ticker = exchange.fetchTicker([symbol])
                    ask_price = ticker['close']
                    balance = exchange.fetchBalance()['total']['USDT']
                    buy_quantity = balance / ask_price
                    buy = exchange.createMarketOrder(symbol=[symbol], side = 'BUY', amount = buy_quantity)
                    print(buy)
                    print(2 * '\n' + 100* '-')
                    print(f'Buy executed for ETH/USDT at SMA: {daily_sma[-1]}, RSI: {exec_tf_rsi[-1]}')
                    print(100 * '-' + '\n')
                    in_position = True
                
            else:
                pnl = exchange.fetchPositions(symbols=[symbol])[-1]['unrealizedPnl']
                print(f'Already in position, PnL = {pnl}')

            if in_position:
                if exec_tf_rsi[-1] > rsi_overbought:
                    sell_qty = exchange.fetchMyTrades(symbol=[symbol])[-1]['info']['qty']
                    sell = exchange.createMarketOrder(symbol=[symbol], side = 'SELL', amount = sell_qty)
                    print(sell)
                    print(2 * '\n' + 100* '-')
                    print(f'Sell executed for {symbol} at SMA: {daily_sma[-1]}, RSI: {exec_tf_rsi[-1]}')
                    print(100 * '-' + '\n')
                    in_position = False
            
            else:
                print('Scanning for Trades')

def run_bot():
    MeanReversionBot()

schedule.every(10).seconds.do(run_bot)

while True:
    schedule.run_pending()
    time.sleep(1)


KeyboardInterrupt: 

In [2]:
import ccxt
import credentials

exchange = ccxt.binanceusdm({
    'enableRateLimit': True,
    'apiKey': credentials.api_key,
    'secret': credentials.api_secret
})

In [19]:
pos = exchange.fetchPositions(symbols=['ETH/USDT'])[-1]
pos

{'info': {'symbol': 'ETHUSDT',
  'positionAmt': '0.029',
  'entryPrice': '1851.6',
  'markPrice': '1844.26209608',
  'unRealizedProfit': '-0.21279921',
  'liquidationPrice': '0',
  'leverage': '1',
  'maxNotionalValue': '3.0E8',
  'marginType': 'cross',
  'isolatedMargin': '0.00000000',
  'isAutoAddMargin': 'false',
  'positionSide': 'BOTH',
  'notional': '53.48360078',
  'isolatedWallet': '0',
  'updateTime': '1686241198465'},
 'id': None,
 'symbol': 'ETH/USDT:USDT',
 'contracts': 0.029,
 'contractSize': 1.0,
 'unrealizedPnl': -0.21279921,
 'leverage': 1.0,
 'liquidationPrice': None,
 'collateral': 53.6964,
 'notional': 53.48360078,
 'markPrice': 1844.26209608,
 'entryPrice': 1851.6,
 'timestamp': 1686241198465,
 'initialMargin': 53.48360078,
 'initialMarginPercentage': 1.0,
 'maintenanceMargin': 0.2674180039,
 'maintenanceMarginPercentage': 0.005,
 'marginRatio': 0.005,
 'datetime': '2023-06-08T16:19:58.465Z',
 'marginMode': 'cross',
 'marginType': 'cross',
 'side': 'long',
 'hedged'

In [23]:
sell_qty = exchange.fetchMyTrades('ETH/USDT')[-1]
sell_qty

{'info': {'symbol': 'ETHUSDT',
  'id': '3045821950',
  'orderId': '8389765602132786192',
  'side': 'SELL',
  'price': '1843.47',
  'qty': '0.026',
  'realizedPnl': '-0.02439000',
  'marginAsset': 'USDT',
  'quoteQty': '47.93022',
  'commission': '0.01917208',
  'commissionAsset': 'USDT',
  'time': '1686248668734',
  'positionSide': 'BOTH',
  'maker': False,
  'buyer': False},
 'timestamp': 1686248668734,
 'datetime': '2023-06-08T18:24:28.734Z',
 'symbol': 'ETH/USDT:USDT',
 'id': '3045821950',
 'order': '8389765602132786192',
 'type': None,
 'side': 'sell',
 'takerOrMaker': 'taker',
 'price': 1843.47,
 'amount': 0.026,
 'cost': 47.93022,
 'fee': {'cost': 0.01917208, 'currency': 'USDT'},
 'fees': [{'cost': 0.01917208, 'currency': 'USDT'}]}

In [21]:
pos = exchange.fetchPositions(symbols=['ETH/USDT'])[-1]
pos

{'info': {'symbol': 'ETHUSDT',
  'positionAmt': '-0.023',
  'entryPrice': '1843.47',
  'markPrice': '1844.29000000',
  'unRealizedProfit': '-0.01886000',
  'liquidationPrice': '4181.30111443',
  'leverage': '1',
  'maxNotionalValue': '3.0E8',
  'marginType': 'cross',
  'isolatedMargin': '0.00000000',
  'isAutoAddMargin': 'false',
  'positionSide': 'BOTH',
  'notional': '-42.41867000',
  'isolatedWallet': '0',
  'updateTime': '1686248668734'},
 'id': None,
 'symbol': 'ETH/USDT:USDT',
 'contracts': 0.023,
 'contractSize': 1.0,
 'unrealizedPnl': -0.01886,
 'leverage': 1.0,
 'liquidationPrice': 4181.30111443,
 'collateral': 54.25096526,
 'notional': 42.41867,
 'markPrice': 1844.29,
 'entryPrice': 1843.47,
 'timestamp': 1686248668734,
 'initialMargin': 42.41867,
 'initialMarginPercentage': 1.0,
 'maintenanceMargin': 0.21209335,
 'maintenanceMarginPercentage': 0.005,
 'marginRatio': 0.0039,
 'datetime': '2023-06-08T18:24:28.734Z',
 'marginMode': 'cross',
 'marginType': 'cross',
 'side': 'sho

In [3]:
exchange.has

{'publicAPI': True,
 'privateAPI': True,
 'CORS': None,
 'spot': False,
 'margin': False,
 'swap': True,
 'future': True,
 'option': None,
 'addMargin': True,
 'cancelAllOrders': True,
 'cancelOrder': True,
 'cancelOrders': None,
 'createDepositAddress': False,
 'createLimitOrder': True,
 'createMarketOrder': True,
 'createOrder': True,
 'createPostOnlyOrder': True,
 'createReduceOnlyOrder': True,
 'createStopOrder': True,
 'createStopLimitOrder': True,
 'createStopMarketOrder': True,
 'editOrder': True,
 'fetchAccounts': None,
 'fetchBalance': True,
 'fetchBidsAsks': True,
 'fetchBorrowInterest': True,
 'fetchBorrowRate': True,
 'fetchBorrowRateHistory': True,
 'fetchBorrowRatesPerSymbol': False,
 'fetchBorrowRates': False,
 'fetchCanceledOrders': 'emulated',
 'fetchClosedOrder': False,
 'fetchClosedOrders': 'emulated',
 'fetchCurrencies': True,
 'fetchDeposit': False,
 'fetchDepositAddress': True,
 'fetchDepositAddresses': False,
 'fetchDepositAddressesByNetwork': False,
 'fetchDepos

In [4]:
import pandas as pd
import numpy as np
from binance.client import Client
import ccxt
from scipy.stats import pearsonr
import credentials

client = Client()
info = client.get_exchange_info()
symbols = [x['symbol'] for x in info['symbols']]
relevant = [symbol for symbol in symbols if symbol.endswith('USDT')]

def get_daily_data(symbol):
    frame = pd.DataFrame(client.get_historical_klines(symbol, '1d', '365 days ago UTC'))
    if len(frame) > 0:
        frame = frame.iloc[:, :6]
        frame.columns = ['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume']
        frame = frame.set_index('Timestamp')
        frame.index = pd.to_datetime(frame.index, unit='ms')
        frame = frame.astype(float)
        return frame

dfs = []

for coin in relevant:
    dfs.append(get_daily_data(coin))

merged_df = pd.concat(dict(zip(relevant, dfs)), axis=1)
close_df = merged_df.loc[:, merged_df.columns.get_level_values(1).isin(['Close'])]
close_df.columns = close_df.columns.droplevel(1)
log_returns_df = np.log(close_df.pct_change() + 1)
coin_correl = log_returns_df.corr()
print(coin_correl)


            BTCUSDT   ETHUSDT   BNBUSDT   NEOUSDT   LTCUSDT  QTUMUSDT   
BTCUSDT    1.000000  0.882950  0.798513  0.723260  0.745845  0.781963  \
ETHUSDT    0.882950  1.000000  0.790607  0.706638  0.763185  0.771057   
BNBUSDT    0.798513  0.790607  1.000000  0.662285  0.692710  0.735836   
NEOUSDT    0.723260  0.706638  0.662285  1.000000  0.635086  0.854873   
LTCUSDT    0.745845  0.763185  0.692710  0.635086  1.000000  0.714436   
...             ...       ...       ...       ...       ...       ...   
PEPEUSDT   0.568435  0.643930  0.516398  0.532758  0.557874  0.543324   
FLOKIUSDT  0.685728  0.765435  0.725080  0.810627  0.787537  0.844719   
ASTUSDT    0.386466  0.398108  0.326201  0.316833  0.379268  0.304350   
SNTUSDT    0.760879  0.804444  0.789987  0.736261  0.901954  0.836268   
COMBOUSDT  0.792317  0.853938  0.699493  0.829297  0.903506  0.840850   

            ADAUSDT   XRPUSDT   EOSUSDT  TUSDUSDT  ...  RDNTUSDT  WBTCUSDT   
BTCUSDT    0.772013  0.680272  0.691120  0.03

In [5]:
coin_correl.to_csv('coin_correlation.csv')

In [ ]:
print